<a href="https://colab.research.google.com/github/JiteshArul/Reddit_Analytics/blob/main/Reddit_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation of required packages

In [1]:
!pip install fake_useragent
!pip install asyncpraw
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 KB 13.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 6.5 MB/s eta 0:00:00


## Importing Required packages

In [2]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import json
import pandas as pd
import praw
import asyncpraw

## Creating a function to scrape the post links from reddit

In [3]:
ua = UserAgent()
#Get number of links to posts of Reddit
def get_reddit_links(html_link):

  last_item=html_link
  linklist=[]

  #Defining number of pages to scrape
  for i in range(10):
    html= requests.get(f"{last_item}", headers={'User-Agent': ua.chrome})
    soup = BeautifulSoup(html.text, 'lxml')
    links=soup.findAll('a', {'class': 'bylink'},href=True)

    # Using next page link for next iteration
    next_page=soup.findAll('a',{'rel': 'nofollow next'})
    last_item=next_page[0]['href']

    #saving each link in a variable
    for link in links:
      linklist.append(link['href'])

  return linklist


In [16]:
#Creating reddit instance
reddit = asyncpraw.Reddit(
    client_id="_gNnMvkwX3z-f8L67pfMxA",
    client_secret="61c3tgEfvSTZwFAG9X2vox3a864Xuw",
    user_agent="Comment Extraction (by u/Jitesh)"
)

#Getting comments from Reddit API
async def comment_extract(url):
  submission = await reddit.submission(url=url)

  #limiting to only 100 sub responses 
  await submission.comments.replace_more(limit=100)
  comment_queue = submission.comments[:]  # Seed with top-level
  comment_list=[]

  while comment_queue:
      comment = comment_queue.pop(0)

      #Creating a list of required parameters from reddit
      comment_list.append([comment.total_awards_received,comment.downs,comment.ups,comment.author,comment.body])
      comment_queue.extend(comment.replies)
      
  return comment_list

[[5, 0, 15907, Redditor(name='Successful_Gate84'), 'Russian roulette'], [3, 0, 30103, Redditor(name='goofandaspoof'), 'That cave exploration hobby where people squeeze through tight spaces. Just fuck no.'], [1, 0, 13744, Redditor(name='SnooBunnies9221'), 'Posting a crying selfie on social media'], [0, 0, 203, Redditor(name='_______woohoo'), 'I dont think I will ever take a fist in my ass'], [0, 0, 9300, Redditor(name='Asleep_Scallion7352'), 'Cave diving. Nope nope nope.'], [0, 0, 2290, Redditor(name='AdMoney9265'), 'Mobile games in YT ads'], [0, 0, 1293, Redditor(name='savelol'), 'Heroin - I work in healthcare, and man oh man does that stuff destroy families and ruin lives. Had a former CEO of a multi-million gaming company who was living in 3 feet of rubbish, pots and pans filled with cigarette butts, urine and shit. He was happy as larry. Content to just inject every day and steal stuff to fund it. His toilet had a literal mountain of poo piling out of the top of it.   \n\n\nI love m